# GitHub Repository Markdown Tool

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/gist/DSamuelHodge/d07a8848fc2094fe69bf242ab4032f2f/github-markdown.ipynb)

This notebook provides a powerful tool for analyzing GitHub repositories in markdown, generating detailed structural analysis, and calculating token counts for AI/ML projects.

## Features
- Generates comprehensive source tree visualization with file sizes and line counts
- Calculates total token count using OpenAI's tiktoken library
- Provides file type statistics and repository summaries
- Parallel processing for faster analysis
- Detailed code content examination
- Handles multiple programming languages
- Excludes common non-code directories (node_modules, __pycache__, etc.)

## Quick Start
```python
from github_analyzer import analyze_github_repo
# Markdown a repository
analyze_github_repo("https://github.com/username/repository")
```

## Notes and Limitations
- Large repositories may take longer to analyze
- Some private repositories may require authentication
- Token counts are estimates based on OpenAI's tiktoken library
- File size limits prevent processing of very large files (>1MB by default)

## Troubleshooting
[Click Here](https://colab.research.google.com/drive/15Vb_5YgWMtjNjD9w2yDSTORYeSVqD0vm#scrollTo=NevbU64ZIme6&line=6&uniqifier=1)

---

# Let's get started

1. Install Dependencies

In [ ]:
!pip install -q gitpython tiktoken rich

from IPython.display import clear_output
clear_output()

print("✅ Dependencies installed successfully!")

2. Import

In [ ]:
import os
import sys
import git
import logging
import json
import tempfile
import shutil
from datetime import datetime
from pathlib import Path
from typing import Set, List, Dict, Optional
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor
import tiktoken
from git.exc import GitCommandError
import re

print("✅ Github markdown tool imported and ready to use!")

3. Insert your the Github repository or subdirectory.

In [ ]:
#insert repo or subdirectory
github_repo = "https://github.com/username/repo.git"

print("✅ Github repository or subdirectory inserted!")

4. Extensions include the following:

In [ ]:
extensions = [".py", ".ipynb", ".js", ".jsx", ".ts", ".tsx", ".html", ".css", ".java", ".c", ".cpp", ".h", ".cs", ".rb", ".php", ".go", ".rs", ".swift", ".kt", ".scala", ".pl", ".lua", ".r", ".sql", ".sh", ".bat", ".m", ".vb", ".erl", ".ex", ".clj", ".hs", ".s", ".asm", ".ps1", ".groovy", ".f", ".f90", ".lisp", ".lsp", ".fs", ".ml", ".jl"]

print("✅ Extensions included!")

5. Analysis Configuration

In [ ]:
@dataclass
class AnalysisConfig:
    """Configuration for the analysis process."""
    extensions: Set[str]
    exclude_dirs: Set[str]
    max_file_size: int
    token_encoding: str
    max_workers: int
    output_format: str

    @classmethod
    def get_default_config(cls) -> Dict:
        return {
            "extensions": extensions,
            "exclude_dirs": [".git", "node_modules", "__pycache__", ".venv"],
            "max_file_size": 1024 * 1024,  # 1MB
            "token_encoding": "cl100k_base",
            "max_workers": 4,
            "output_format": "markdown"
        }

    @classmethod
    def from_file(cls, config_path: Optional[str] = None) -> 'AnalysisConfig':
        """Load configuration from a JSON file or use defaults."""
        default_config = cls.get_default_config()

        if config_path:
            try:
                with open(config_path, 'r') as f:
                    config = json.load(f)
                    config = {**default_config, **config}
            except (FileNotFoundError, json.JSONDecodeError) as e:
                print(f"Warning: Could not load config file ({str(e)}). Using defaults.")
                config = default_config
        else:
            config = default_config

        return cls(
            extensions=set(config["extensions"]),
            exclude_dirs=set(config["exclude_dirs"]),
            max_file_size=config["max_file_size"],
            token_encoding=config["token_encoding"],
            max_workers=config["max_workers"],
            output_format=config["output_format"]
        )


print("✅ Analysis configuration dataclass is loaded!")

6. Github Analyzer Code

In [ ]:
class GitHubAnalyzer:
    def __init__(self, config: AnalysisConfig):
        self.config = config
        self.token_counter = tiktoken.get_encoding(config.token_encoding)

    def parse_github_url(self, github_url: str) -> tuple:
        """Parse GitHub URL to extract repository URL and subdirectory path."""
        # Remove 'tree' or 'blob' from URL if present
        url_parts = github_url.replace('/tree/', '/').replace('/blob/', '/').split('/')

        # Find the main repository URL (everything up to the branch name)
        repo_parts = []
        branch_found = False
        subdirectory_parts = []

        for i, part in enumerate(url_parts):
            if i < 5:  # Always include the first 5 parts (https://github.com/user/repo)
                repo_parts.append(part)
            elif not branch_found and '.' not in part and part not in ['main', 'master']:
                repo_parts.append(part)
            else:
                branch_found = True
                if i < len(url_parts) - 1:  # Don't include the branch name in subdirectory
                    subdirectory_parts.append(part)

        repo_url = '/'.join(repo_parts)
        subdirectory = '/'.join(subdirectory_parts[1:] if subdirectory_parts else [])

        return repo_url, subdirectory

    def clone_repository(self, github_url: str, target_dir: str) -> Optional[git.Repo]:
        """Clone a GitHub repository."""
        try:
            print(f"Cloning {github_url}...")
            return git.Repo.clone_from(github_url, target_dir)
        except GitCommandError as e:
            print(f"Failed to clone repository: {e}")
            return None

    def process_file(self, file_path: Path, repo_root: Path) -> Dict:
        """Process a single file and return its analysis results."""
        try:
            if file_path.stat().st_size > self.config.max_file_size:
                print(f"Skipping large file: {file_path}")
                return {"skip": True}

            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read()
                return {
                    "content": content,
                    "tokens": len(self.token_counter.encode(content)),
                    "relative_path": str(file_path.relative_to(repo_root)),
                    "extension": file_path.suffix,
                    "skip": False
                }
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")
            return {"skip": True}

    def format_size(self, size_bytes: int) -> str:
        """Format file size in human readable format."""
        for unit in ['B', 'KB', 'MB', 'GB']:
            if size_bytes < 1024:
                return f"{size_bytes:.1f}{unit}"
            size_bytes /= 1024
        return f"{size_bytes:.1f}GB"

    def count_lines(self, content: str) -> int:
        """Count non-empty lines in content."""
        return len([line for line in content.splitlines() if line.strip()])

    def analyze_repository(self, github_url: str) -> bool:
        """Analyze a GitHub repository and generate documentation."""
        if not github_url:
            print("No GitHub URL provided.")
            return False

        # Parse the GitHub URL to handle subdirectories
        repo_url, subdirectory = self.parse_github_url(github_url)
        if subdirectory:
            print(f"Analyzing subdirectory: {subdirectory}")

        # Create temporary directory for cloning
        with tempfile.TemporaryDirectory() as temp_dir:
            repo_path = Path(temp_dir)
            repo = self.clone_repository(repo_url, repo_path)

            if not repo:
                return False

            # Determine the path to analyze
            analysis_path = repo_path / subdirectory if subdirectory else repo_path
            if not analysis_path.exists():
                print(f"Subdirectory {subdirectory} not found in repository")
                return False

            # Extract repository/directory name for output
            repo_name = subdirectory.split('/')[-1] if subdirectory else repo_url.rstrip("/").split("/")[-1].replace(".git", "")

            # Initialize analysis results
            analysis_results = {
                "repository": repo_name,
                "timestamp": datetime.now().isoformat(),
                "files": [],
                "total_tokens": 0
            }

            # Collect files for processing
            files_to_process = []
            for root, dirs, files in os.walk(analysis_path):
                dirs[:] = [d for d in dirs if d not in self.config.exclude_dirs]

                for file in files:
                    file_path = Path(root) / file
                    if any(file.endswith(ext) for ext in self.config.extensions):
                        files_to_process.append(file_path)

            print(f"Processing {len(files_to_process)} files...")

            # Process files in parallel
            with ThreadPoolExecutor(max_workers=self.config.max_workers) as executor:
                futures = [
                    executor.submit(self.process_file, file_path, analysis_path)
                    for file_path in files_to_process
                ]

                for future in futures:
                    result = future.result()
                    if not result["skip"]:
                        analysis_results["files"].append(result)
                        analysis_results["total_tokens"] += result["tokens"]

            # Generate output
            output_path = self.generate_output(analysis_results)
            print(f"Analysis completed. Output saved to: {output_path}")
            return True

    def generate_output(self, analysis_results: Dict) -> str:
        """Generate analysis output in the specified format."""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M")
        output_path = f"{analysis_results['repository']}_analysis_{timestamp}"

        if self.config.output_format == "markdown":
            output_path += ".md"
            self.generate_markdown(analysis_results, output_path)
        elif self.config.output_format == "json":
            output_path += ".json"
            self.generate_json(analysis_results, output_path)
        else:
            print(f"Unsupported output format: {self.config.output_format}")
            return ""

        return output_path

    def generate_markdown(self, analysis_results: Dict, output_path: str):
        """Generate Markdown documentation with enhanced source tree."""
        with open(output_path, 'w', encoding='utf-8') as f:
            # Repository header
            repo_name = analysis_results['repository']
            f.write(f"# {repo_name} Analysis\n\n")

            # Summary section
            total_files = len(analysis_results['files'])
            total_lines = sum(self.count_lines(file['content']) for file in analysis_results['files'])
            total_size = sum(len(file['content'].encode('utf-8')) for file in analysis_results['files'])

            f.write("## Repository Summary\n\n")
            f.write(f"- **Generated:** {analysis_results['timestamp']}\n")
            f.write(f"- **Total Files:** {total_files:,}\n")
            f.write(f"- **Total Lines:** {total_lines:,}\n")
            f.write(f"- **Total Size:** {self.format_size(total_size)}\n")
            f.write(f"- **Total Tokens:** {analysis_results['total_tokens']:,}\n\n")

            # File type statistics
            extension_stats = {}
            for file in analysis_results['files']:
                ext = file['extension']
                if ext not in extension_stats:
                    extension_stats[ext] = {'count': 0, 'lines': 0, 'size': 0}
                stats = extension_stats[ext]
                stats['count'] += 1
                stats['lines'] += self.count_lines(file['content'])
                stats['size'] += len(file['content'].encode('utf-8'))

            f.write("## File Type Statistics\n\n")
            f.write("| Extension | Files | Lines | Size |\n")
            f.write("|-----------|-------|-------|------|\n")
            for ext, stats in sorted(extension_stats.items(), key=lambda x: x[1]['lines'], reverse=True):
                f.write(f"| {ext or 'no ext'} | {stats['count']:,} | {stats['lines']:,} | {self.format_size(stats['size'])} |\n")
            f.write("\n")

            # Enhanced source tree
            f.write("## Source Tree\n\n")
            f.write("```\n")

            # Organize files by directory
            file_tree = {}
            for file_info in analysis_results["files"]:
                path_parts = file_info["relative_path"].split(os.sep)
                current_dict = file_tree
                for part in path_parts[:-1]:
                    if part not in current_dict:
                        current_dict[part] = {'__files__': []}
                    current_dict = current_dict[part]
                current_dict['__files__'] = current_dict.get('__files__', [])
                current_dict['__files__'].append(file_info)

            # Write tree structure with enhanced information
            def write_tree(d: Dict, prefix: str = "", is_last: bool = True):
                entries = sorted([(k, v) for k, v in d.items() if k != '__files__'])
                files = d.get('__files__', [])

                for idx, (name, content) in enumerate(entries):
                    is_last_entry = idx == len(entries) - 1 and not files
                    f.write(f"{prefix}{'└──' if is_last_entry else '├──'} {name}/\n")
                    new_prefix = prefix + ('    ' if is_last_entry else '│   ')
                    write_tree(content, new_prefix, idx == len(entries) - 1)

                for idx, file_info in enumerate(sorted(files, key=lambda x: x['relative_path'])):
                    is_last_file = idx == len(files) - 1
                    name = os.path.basename(file_info['relative_path'])
                    lines = self.count_lines(file_info['content'])
                    size = len(file_info['content'].encode('utf-8'))
                    f.write(f"{prefix}{'└──' if is_last_file else '├──'} {name:<30} "
                           f"[{self.format_size(size):>7}, {lines:>4} lines]\n")

            write_tree(file_tree)
            f.write("```\n\n")

            # File contents
            f.write("## File Contents\n\n")
            for file_info in sorted(analysis_results["files"],
                                  key=lambda x: x["relative_path"]):
                f.write(f"### {file_info['relative_path']}\n\n")
                f.write(f"```{file_info['extension'][1:]}\n")
                f.write(file_info["content"])
                f.write("\n```\n\n")

    def generate_json(self, analysis_results: Dict, output_path: str):
        """Generate JSON output."""
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(analysis_results, f, indent=2)

def analyze_github_repo(github_url: str, config_path: Optional[str] = None):
    """Analyze a GitHub repository from a notebook."""
    config = AnalysisConfig.from_file(config_path)
    analyzer = GitHubAnalyzer(config)
    return analyzer.analyze_repository(github_url)

# Batch analysis function
def batch_analyze_repos(repo_list: List[str], config_path: Optional[str] = None):
    """Analyze multiple repositories in batch."""
    total_repos = len(repo_list)
    successful = []
    failed = []
    start_time = datetime.now()

    print(f"Starting batch analysis of {total_repos} repositories...")
    print("=" * 50)

    for idx, repo_url in enumerate(repo_list, 1):
        try:
            print(f"\n[{idx}/{total_repos}] Analyzing: {repo_url}")
            result = analyze_github_repo(repo_url, config_path)
            if result:
                successful.append(repo_url)
            else:
                failed.append((repo_url, "Analysis failed"))
        except Exception as e:
            failed.append((repo_url, str(e)))
            print(f"Error processing {repo_url}: {e}")

        print("-" * 50)

    # Generate summary report
    duration = datetime.now() - start_time

    print("\nBatch Analysis Summary")
    print("=" * 50)
    print(f"Time taken: {duration}")
    print(f"Total repositories: {total_repos}")
    print(f"Successfully analyzed: {len(successful)}")
    print(f"Failed: {len(failed)}")

    if failed:
        print("\nFailed Repositories:")
        for repo, error in failed:
            print(f"- {repo}: {error}")


print("✅ Github analyzer code is ready to use!")

7. (Optional) Customize Configuration

In [ ]:
config = {
    "extensions": extensions,
    "exclude_dirs": [".git", "node_modules", "__pycache__", ".venv"],
    "max_file_size": 1048576,
    "token_encoding": "cl100k_base",
    "max_workers": 4,
    "output_format": "markdown"
}

with open('config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("✅ Configuration saved to 'config.json'")

## Usage Examples



### 1. Single GitHub Repository Analysis
Analyze a complete repository or specific subdirectory:

In [ ]:
analyze_github_repo(github_repo)

---

### 2. Batch Analysis

This code provides several features:

1. **Batch Processing:**
   - Processes multiple repositories sequentially
   - Tracks successful and failed analyses
   - Provides progress updates

2. **Error Handling:**
   - Catches and logs errors for each repository
   - Continues processing even if one repository fails
   - Provides detailed error messages in the summary

3. **Performance Tracking:**
   - Measures total execution time
   - Shows progress (current/total repositories)
   - Generates a summary report

4. **Flexibility:**
   - Can take a list of repositories directly
   - Can read repositories from a text file
   - Easy to modify for different input formats

You can use it in two ways:

1. **Direct list:**
```python
repos = [
    "https://github.com/repo1",
    "https://github.com/repo2",
    "https://github.com/repo3"
]
batch_analyze_repos(repos)
```

2. **From file:**
```python
# First create the file
%%writefile repos.txt
https://github.com/repo1
https://github.com/repo2
https://github.com/repo3
# Then analyze
repos = read_repos_from_file('repos.txt')
batch_analyze_repos(repos)
```

The output will show progress for each repository and provide a summary at the end with success/failure statistics and timing information.

**2.1 Direct List**

In [ ]:
repos_to_analyze = [
    "https://github.com/repo1",
    "https://github.com/repo2",
    "https://github.com/repo3",
    # Add more repositories here
]

batch_analyze_repos(repos_to_analyze)

**2.2 Read repositories from file**

First Create the File

In [ ]:
# First create the file
%%writefile repos.txt
https://github.com/repo1
https://github.com/repo2
https://github.com/repo3

Read the file and analyze the batch repositories

In [ ]:
def read_repos_from_file(filename):
    """Read repository URLs from a text file."""
    with open(filename, 'r') as f:
        return [line.strip() for line in f if line.strip()]

repos = read_repos_from_file('repos.txt')
batch_analyze_repos(repos)

print("✅ Repositories read from file and batch analysis completed!")

---

### 3. Interactive Analysis

Run and Enter your GitHub Repo URL

In [ ]:
def interactive_analysis():
    """Interactive repository analysis."""
    while True:
        repo_url = input("Enter GitHub repository URL (or 'q' to quit): ").strip()
        if repo_url.lower() == 'q':
            print("Analysis completed.")
            break
        if repo_url:
            print(f"\nAnalyzing repository: {repo_url}")
            analyze_github_repo(repo_url)
            print("\n---\n")

# Example usage:
interactive_analysis()

---

## Troubleshooting
If you encounter issues:
1. Check the repository URL format
2. Ensure the repository is public
3. Verify subdirectory paths if analyzing specific directories
4. Check your internet connection for cloning issues

## Example Usage Documentation

In [17]:
def show_examples():
    print("Example Usage:")
    print("\n1. Analyze a complete repository:")
    print('analyze_github_repo("https://github.com/username/repository")')

    print("\n2. Analyze a specific subdirectory:")
    print('analyze_github_repo("https://github.com/username/repository/tree/main/src")')

    print("\n3. Batch analysis:")
    print('repos = [\n    "https://github.com/repo1",\n    "https://github.com/repo2"\n]')
    print('batch_analyze_repos(repos)')

# Uncomment to show examples
show_examples()

Example Usage:

1. Analyze a complete repository:
analyze_github_repo("https://github.com/username/repository")

2. Analyze a specific subdirectory:
analyze_github_repo("https://github.com/username/repository/tree/main/src")

3. Batch analysis:
repos = [
    "https://github.com/repo1",
    "https://github.com/repo2"
]
batch_analyze_repos(repos)
